In [1]:
# calculate priorization of predicted efficac for each cell lines
import pandas as pd
import numpy as np
from glob import glob

In [ ]:
# read aoc_pred_0.txt
# read ../../test_by_cell_line/fold_0/Text.tsv

In [17]:
# read prediction of all folds and merge with orginal test data
score = 'bliss'
split = 'test_by_cell_line'
all_path = glob('../'+split+'/fold*')
model_path = '../experiment_'+split+'/features_monotherapy_moa_geneset_molecular_synthetic_lethality_target_gene_network_chemical_structure_drug_name'
all_data = []
for path in all_path:
    fold = path.split('/')[-1].split('_')[-1]
    print(fold)
    data = pd.read_csv(path+'/Test.tsv', sep='\t')
    pred = pd.read_csv(model_path+'/aoc_pred_'+fold+'.txt', sep=' ', header=None, names=['.response_'+score, '.response_'+score+'_pred'])
    data['.response_'+score+'_pred'] = pred['.response_'+score+'_pred']
    all_data.append(data)
all_data = pd.concat(all_data)

# add hold_out 
data = pd.read_csv('../'+split+'/hold_out_validation/Test.tsv', sep='\t')
# read all predictions on hold_out and get the mean
pred = []
tmp_path = glob(model_path+'/hold_out/Test/'+score+'_pred_*.txt')
for path in tmp_path:
    tmp = np.loadtxt(path)
    pred.append(tmp)
# get the mean
pred = np.mean(np.array(pred), axis=0)
data['.response_'+score+'_pred'] = pred[:,1]

all_data = pd.concat([all_data, data])

# add hold_out for new tissues
split = 'test_by_indication'
model_path = '../experiment_'+split+'/features_monotherapy_moa_geneset_molecular_synthetic_lethality_target_gene_network_chemical_structure_drug_name'
for tissue in ['cervix', 'kidney']:
    data = pd.read_csv('../'+split+'/hold_out_validation/Test_'+tissue+'.tsv', sep='\t')
    # read all predictions on hold_out and get the mean
    pred = []
    tmp_path = glob(model_path+'/hold_out/Test_'+tissue+'/'+score+'_pred_*.txt')
    for path in tmp_path:
        tmp = np.loadtxt(path)
        pred.append(tmp)
    # get the mean
    pred = np.mean(np.array(pred), axis=0)
    data['.response_'+score+'_pred'] = pred[:,1]
    all_data = pd.concat([all_data, data])

all_data['combination'] = ['.'.join(sorted([r['.metadata_treatment_1'],r['.metadata_treatment_2']])) for i,r in all_data.iterrows()]
# moa combination
all_data['moa_combination'] = ['.'.join(sorted([r['.metadata_moa_1'],r['.metadata_moa_2']])) for i,r in all_data.iterrows()]
# DDR partner type: either moa is ATMi/ATRi/DNAPKi
all_data['ddr_partner_type'] = ['ATMi' if r['.metadata_moa_1'] == 'ATMi' or r['.metadata_moa_2'] == 'ATMi' else 'ATRi' if r['.metadata_moa_1'] == 'ATRi' or r['.metadata_moa_2'] == 'ATRi' else 'DNAPKi' if r['.metadata_moa_1'] == 'DNAPKi' or r['.metadata_moa_2'] == 'DNAPKi' else 'other' for i,r in all_data.iterrows()]


1
0
4
7
8
6
9
3
5
2


In [23]:
# prioritizing on the cell lines
improve_df = { 'cell_line':[], 'tissue':[], 'median_efficacy':[], 'selected_efficacy':[], 'improvement':[], 'improvement_numeric':[]}
all_improvement = []

all_cell_lines = all_data['.identifier_sample_name'].unique()
for cell_line in all_cell_lines:
    sub_data = all_data.loc[all_data['.identifier_sample_name'] == cell_line,:]
    tissue = sub_data['.metadata_cancer_type'].iloc[0]
    print(tissue)
    ave_efficacy = sub_data['.response_'+score].median()
    top_efficacy = sub_data['.response_'+score+'_pred'].max()
    improvement = (top_efficacy - ave_efficacy)/(np.abs(ave_efficacy))
    improvement_per = "{0:.0%}".format(improvement)
    improve_df['tissue'].append(tissue)
    improve_df['cell_line'].append(cell_line)
    improve_df['median_efficacy'].append(ave_efficacy)
    improve_df['selected_efficacy'].append(top_efficacy)
    improve_df['improvement'].append(improvement_per)
    improve_df['improvement_numeric'].append(improvement)
    if improvement>=0:
        all_improvement.append(1)
    else:
        all_improvement.append(0)

print("total_improvement",np.mean(np.array(all_improvement)))
improve_df = pd.DataFrame.from_dict(improve_df)
# sort by improvement
improve_df = improve_df.sort_values(by='improvement_numeric', ascending=False)
# drop column 'improvement_numeric'
improve_df = improve_df.drop(columns=['improvement_numeric'])
improve_df.to_csv("improvemt_"+score+".csv", index =False)

ovary
pancreas
breast
sarcoma
brain
colon
hematological
pancreas
ovary
colon
brain
breast
sarcoma
hematological
colon
sarcoma
bladder
breast
ovary
hematological
colon
lung
sarcoma
melanoma
bladder
hematological
lung
colon
liver
sarcoma
melanoma
hematological
colon
bladder
lung
sarcoma
melanoma
hematological
colon
sarcoma
lung
liver
brain
hematological
prostate
breast
pancreas
ovary
sarcoma
hematological
colon
breast
lung
bladder
sarcoma
hematological
sarcoma
ovary
breast
pancreas
prostate
hematological
prostate
bladder
melanoma
sarcoma
breast
colon
colon
breast
sarcoma
pancreas
liver
ovary
brain
lung
hematological
hematological
cervix
cervix
cervix
kidney
kidney
kidney
kidney
kidney
total_improvement 1.0


/tmp/ipykernel_53035/506159721.py:12: RuntimeWarning: divide by zero encountered in scalar divide
  improvement = (top_efficacy - ave_efficacy)/(np.abs(ave_efficacy))


In [21]:
improve_df = { 'combination':[], 'moa_combination':[],'median_efficacy':[], 'selected_efficacy':[], 'improvement':[], 'improvement_numeric':[], 'pearson':[]}
all_improvement = []

import numpy.ma as ma

all_comb = all_data['combination'].unique()
for comb in all_comb:
    sub_data = all_data.loc[all_data['combination'] == comb,:]
    #tissue = sub_data['.metadata_cancer_type'].iloc[0]
    #print(tissue)
    ave_efficacy = sub_data['.response_'+score].median(skipna=True)
    top_efficacy = sub_data['.response_'+score+'_pred'].max()

    improvement = (top_efficacy - ave_efficacy)/(np.abs(ave_efficacy))
    improvement_per = "{0:.0%}".format(improvement)
    improve_df['combination'].append(comb)
    improve_df['moa_combination'].append(sub_data['moa_combination'].iloc[0])
    improve_df['median_efficacy'].append(ave_efficacy)
    improve_df['selected_efficacy'].append(top_efficacy)
    improve_df['improvement'].append(improvement_per)
    improve_df['improvement_numeric'].append(improvement)
    # pearson's correlation between predicted and observed
    pearson =  cor = ma.corrcoef(ma.masked_invalid(sub_data['.response_'+score]), ma.masked_invalid(sub_data['.response_'+score+'_pred']))[0,1] # ignore nan
    improve_df['pearson'].append(pearson)
    if improvement>=0:
        all_improvement.append(1)
    else:
        all_improvement.append(0)

print("total_improvement",np.mean(np.array(all_improvement)))
improve_df = pd.DataFrame.from_dict(improve_df)
# sort by improvement
improve_df = improve_df.sort_values(by='pearson', ascending=False)
# drop column 'improvement_numeric'
improve_df = improve_df.drop(columns=['improvement_numeric'])
improve_df.to_csv("improvemt_efficacy_combinations_"+score+".csv", index =False)

/tmp/ipykernel_53035/3578182693.py:14: RuntimeWarning: divide by zero encountered in scalar divide
  improvement = (top_efficacy - ave_efficacy)/(np.abs(ave_efficacy))


total_improvement 0.9937238493723849


In [20]:
all_improvement = []
all_comb = all_data['moa_combination'].unique()
improve_df = { 'combination':[],'median_efficacy':[], 'selected_efficacy':[], 'improvement':[], 'improvement_numeric':[], 'pearson':[]}
for comb in all_comb:
    sub_data = all_data.loc[all_data['moa_combination'] == comb,:]
    #tissue = sub_data['.metadata_cancer_type'].iloc[0]
    #print(tissue)
    ave_efficacy = sub_data['.response_'+score].median(skipna=True)
    top_efficacy = sub_data['.response_'+score+'_pred'].max()

    improvement = (top_efficacy - ave_efficacy)/(np.abs(ave_efficacy))
    improvement_per = "{0:.0%}".format(improvement)
    improve_df['combination'].append(comb)
    #improve_df['moa_combination'].append(sub_data['moa_combination'].iloc[0])
    improve_df['median_efficacy'].append(ave_efficacy)
    improve_df['selected_efficacy'].append(top_efficacy)
    improve_df['improvement'].append(improvement_per)
    improve_df['improvement_numeric'].append(improvement)
    # pearson's correlation between predicted and observed
    pearson =  cor = ma.corrcoef(ma.masked_invalid(sub_data['.response_'+score]), ma.masked_invalid(sub_data['.response_'+score+'_pred']))[0,1] # ignore nan
    improve_df['pearson'].append(pearson)
    if improvement>=0:
        all_improvement.append(1)
    else:
        all_improvement.append(0)

print("total_improvement",np.mean(np.array(all_improvement)))
improve_df = pd.DataFrame.from_dict(improve_df)
# sort by improvement
improve_df = improve_df.sort_values(by='pearson', ascending=False)
# drop column 'improvement_numeric'
improve_df = improve_df.drop(columns=['improvement_numeric'])
improve_df.to_csv("improvemt_efficacy_moa_"+score+".csv", index =False)

total_improvement 0.9878048780487805


/tmp/ipykernel_53035/1869623247.py:11: RuntimeWarning: divide by zero encountered in scalar divide
  improvement = (top_efficacy - ave_efficacy)/(np.abs(ave_efficacy))


In [19]:
all_improvement = []
all_comb = all_data['ddr_partner_type'].unique()
improve_df = { 'combination':[],'median_efficacy':[], 'selected_efficacy':[], 'improvement':[], 'improvement_numeric':[], 'pearson':[]}
for comb in all_comb:
    sub_data = all_data.loc[all_data['ddr_partner_type'] == comb,:]
    #tissue = sub_data['.metadata_cancer_type'].iloc[0]
    #print(tissue)
    ave_efficacy = sub_data['.response_'+score].median(skipna=True)
    top_efficacy = sub_data['.response_'+score+'_pred'].max()

    improvement = (top_efficacy - ave_efficacy)/(np.abs(ave_efficacy))
    improvement_per = "{0:.0%}".format(improvement)
    improve_df['combination'].append(comb)
    #improve_df['moa_combination'].append(sub_data['moa_combination'].iloc[0])
    improve_df['median_efficacy'].append(ave_efficacy)
    improve_df['selected_efficacy'].append(top_efficacy)
    improve_df['improvement'].append(improvement_per)
    improve_df['improvement_numeric'].append(improvement)
    # pearson's correlation between predicted and observed
    pearson =  cor = ma.corrcoef(ma.masked_invalid(sub_data['.response_'+score]), ma.masked_invalid(sub_data['.response_'+score+'_pred']))[0,1] # ignore nan
    improve_df['pearson'].append(pearson)
    if improvement>=0:
        all_improvement.append(1)
    else:
        all_improvement.append(0)

print("total_improvement",np.mean(np.array(all_improvement)))
improve_df = pd.DataFrame.from_dict(improve_df)
# sort by improvement
improve_df = improve_df.sort_values(by='pearson', ascending=False)
# drop column 'improvement_numeric'
improve_df = improve_df.drop(columns=['improvement_numeric'])
improve_df.to_csv("improvemt_efficacy_ddr_partner_"+score+".csv", index =False)



total_improvement 0.75
